In [16]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
from skimage.io import imread

import sys
import os
import glob
import shutil
import argparse
from tqdm import tqdm

from albumentations import (
    convert_bbox_to_albumentations,convert_bboxes_from_albumentations
)

import albumentations as A

ModuleNotFoundError: No module named 'albumentations'

In [2]:
!pip install tqdm

     |████████████████████████████████| 63 kB 456 kB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [17]:
!pip install albumentations

     |████████████████████████████████| 116 kB 149 kB/s eta 0:00:01
     |████████████████████████████████| 631 kB 162 kB/s eta 0:00:01
     |████████████████████████████████| 269 kB 74 kB/s eta 0:00:01
  Created wheel for albumentations: filename=albumentations-0.4.5-py3-none-any.whl size=70248 sha256=3012e0b905eee50cd167661e3613ad341d487dc4640b6b03168369bc6c6af807
  Stored in directory: /root/.cache/pip/wheels/78/d2/b4/617573279167b9cb8fd3a2d17923ae044d5d213bb552abc1be
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654972 sha256=3a1d3a6dabe013f3f09b3eff2a204b64b393e20db7cec4e3ac6b5d0f9280bae4
  Stored in directory: /root/.cache/pip/wheels/ce/b4/ea/ed7e89421721d66834627a2983c5bc4ce298056d5d3f37049e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=45919 sha256=dea189e621586c02dde9d7920790a460e5e41ab9d868a932286bc2092fa71667
  Stored in directory: /root/.cache/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80f

In [3]:
root_path = "/tf/home/sergio/Tesis"

In [5]:
#Leemos los archivos CSV del VOC Dataset

os.chdir(root_path+"/VOC2012")
voc_annotations = pd.read_csv("VOC_annotations.csv")
voc_metadata = pd.read_csv("VOC_metadata.csv",index_col=0)
print(voc_annotations.shape)
print(voc_metadata.shape)
display(voc_annotations.head())
display(voc_metadata.head())

(17401, 5)
(9583, 2)


,file_name,xmin,ymin,xmax,ymax
0,2011_004953.jpg,0.048048,0.034000,0.903904,1.000
1,2008_006944.jpg,0.384000,0.288000,0.582000,0.832
2,2008_006944.jpg,0.458000,0.240000,0.620000,0.576
3,2011_000700.jpg,0.570455,0.179775,0.872727,1.000
4,2011_000700.jpg,0.263636,0.059925,0.581818,1.000


,width,height
2011_004953.jpg,333,500
2008_006944.jpg,500,375
2011_000700.jpg,440,267
2010_006541.jpg,500,375
2011_004154.jpg,500,375


In [7]:
#Leemos los archivos CSV del COCO Dataset

os.chdir(root_path+"/COCODataset/coco_csvs")
coco_annotations_train = pd.read_csv("COCO_annotations_train.csv", index_col=0)
coco_metadata_train = pd.read_csv("COCO_metadata_train.csv", index_col=0)

coco_annotations_val = pd.read_csv("COCO_annotations_val.csv", index_col=0)
coco_metadata_val = pd.read_csv("COCO_metadata_val.csv", index_col=0)

print(coco_annotations_train.shape)
print(coco_metadata_train.shape)
display(coco_annotations_train.head())
display(coco_metadata_train.head())

print(coco_annotations_val.shape)
print(coco_metadata_val.shape)
display(coco_annotations_val.head())
display(coco_metadata_val.head())

(262461, 5)
(64115, 2)


,file_name,xmin,ymin,xmax,ymax
0,000000262145.jpg,0.604688,0.166511,0.831250,0.920609
1,000000262145.jpg,0.772359,0.213208,0.790641,0.251054
2,000000262145.jpg,0.441203,0.203255,0.491453,0.430468
3,000000262145.jpg,0.822141,0.103372,0.963547,0.543653
4,000000262145.jpg,0.767984,0.190141,0.831203,0.390000


,width,height
000000262145.jpg,640,427
000000262146.jpg,480,640
000000524291.jpg,640,426
000000262148.jpg,640,512
000000393223.jpg,640,480


(11004, 5)
(2693, 2)


,file_name,xmin,ymin,xmax,ymax
0,000000532481.jpg,0.391906,0.394977,0.501453,0.547277
1,000000532481.jpg,0.446172,0.869859,0.465031,0.914953
2,000000458755.jpg,0.107859,0.078646,0.901687,0.986521
3,000000458755.jpg,0.885938,0.002646,1.000000,0.183604
4,000000458755.jpg,0.921906,0.191021,0.999437,0.393250


,width,height
000000532481.jpg,640,426
000000458755.jpg,640,480
000000385029.jpg,640,480
000000311303.jpg,640,427
000000393226.jpg,640,480


Juntando los CSV de VOC Dtaset y COCO Dataset

In [8]:
frames = [coco_annotations_train,voc_annotations]
annotations_train_df = pd.concat(frames)

frames = [coco_metadata_train,voc_metadata]
metadata_train_df = pd.concat(frames)

In [9]:
annotations_train_df

,file_name,xmin,ymin,xmax,ymax
0,000000262145.jpg,0.604688,0.166511,0.831250,0.920609
1,000000262145.jpg,0.772359,0.213208,0.790641,0.251054
2,000000262145.jpg,0.441203,0.203255,0.491453,0.430468
3,000000262145.jpg,0.822141,0.103372,0.963547,0.543653
4,000000262145.jpg,0.767984,0.190141,0.831203,0.390000
...,...,...,...,...,...
17396,2008_000916.jpg,0.890000,0.504000,0.958000,0.717333
17397,2008_000916.jpg,0.888000,0.517333,0.926000,0.605333
17398,2012_000065.jpg,0.002000,0.264000,0.814000,1.000000
17399,2009_002407.jpg,0.304000,0.132132,0.532000,0.810811


In [22]:
annotations_train_df.loc[:,"file_name"].value_counts()

000000497261.jpg    20
2008_001825.jpg     18
2008_006265.jpg     18
000000427154.jpg    18
2008_003819.jpg     18
                    ..
000000249515.jpg     1
000000169028.jpg     1
000000043324.jpg     1
000000132527.jpg     1
000000226939.jpg     1
Name: file_name, Length: 73698, dtype: int64

In [23]:
#Obtenemos el número máximo de objetos en una sola imagen
image_name = annotations_train_df.file_name.mode().values[0]
max_objects = len(annotations_train_df[annotations_train_df.file_name==image_name])

In [25]:
max_objects

20

In [26]:
metadata_train_df

,width,height
000000262145.jpg,640,427
000000262146.jpg,480,640
000000524291.jpg,640,426
000000262148.jpg,640,512
000000393223.jpg,640,480
...,...,...
2010_006788.jpg,500,375
2012_002969.jpg,500,333
2008_000916.jpg,500,375
2012_000065.jpg,500,375


In [27]:
boxes = annotations_train_df[annotations_train_df.file_name =="000000262145.jpg"].iloc[:,1:].values
print(boxes)
dummy_box = [[0.4,0.4,0.5,0.5]]
ux= np.repeat(dummy_box,repeats = 20-boxes.shape[0],axis=0)
print(ux)

np.concatenate((boxes,ux),axis=0)

[[0.6046875  0.16651054 0.83125    0.9206089 ]
 [0.77235937 0.21320843 0.79064062 0.25105386]
 [0.44120312 0.20325527 0.49145312 0.43046838]
 [0.82214062 0.10337237 0.96354688 0.5436534 ]
 [0.76798437 0.19014052 0.83120313 0.39      ]
 [0.7466875  0.20562061 0.76809375 0.28384075]
 [0.48278125 0.17976581 0.56073438 0.52807963]
 [0.11245313 0.24494145 0.20540625 0.5842623 ]
 [0.55092188 0.31885246 0.59490625 0.39107728]
 [0.08153125 0.23030445 0.11573438 0.30934426]
 [0.06278125 0.22166276 0.08457812 0.27447307]
 [0.00515625 0.28400468 0.13521875 0.57562061]
 [0.23934375 0.14501171 0.4061875  0.42093677]
 [0.040625   0.1498829  0.9984375  0.44496487]]
[[0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]]


array([[0.6046875 , 0.16651054, 0.83125   , 0.9206089 ],
       [0.77235937, 0.21320843, 0.79064062, 0.25105386],
       [0.44120312, 0.20325527, 0.49145312, 0.43046838],
       [0.82214062, 0.10337237, 0.96354688, 0.5436534 ],
       [0.76798437, 0.19014052, 0.83120313, 0.39      ],
       [0.7466875 , 0.20562061, 0.76809375, 0.28384075],
       [0.48278125, 0.17976581, 0.56073438, 0.52807963],
       [0.11245313, 0.24494145, 0.20540625, 0.5842623 ],
       [0.55092188, 0.31885246, 0.59490625, 0.39107728],
       [0.08153125, 0.23030445, 0.11573438, 0.30934426],
       [0.06278125, 0.22166276, 0.08457812, 0.27447307],
       [0.00515625, 0.28400468, 0.13521875, 0.57562061],
       [0.23934375, 0.14501171, 0.4061875 , 0.42093677],
       [0.040625  , 0.1498829 , 0.9984375 , 0.44496487],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       ,

In [14]:
os.chdir(root_path+"/TinyYOLOv3-Pedestrian-Detection")
annotations_train_df.to_csv("annotations_train_df.csv",index=False)
metadata_train_df.to_csv("metadata_train_df.csv",index=False)

In [28]:
COCO_metadata_train.head()

NameError: name 'COCO_metadata_train' is not defined

In [29]:
coco_metadata_train

,width,height
000000262145.jpg,640,427
000000262146.jpg,480,640
000000524291.jpg,640,426
000000262148.jpg,640,512
000000393223.jpg,640,480
...,...,...
000000524273.jpg,640,427
000000131058.jpg,640,480
000000393207.jpg,640,359
000000262136.jpg,500,375


In [30]:
voc_metadata

,width,height
2011_004953.jpg,333,500
2008_006944.jpg,500,375
2011_000700.jpg,440,267
2010_006541.jpg,500,375
2011_004154.jpg,500,375
...,...,...
2010_006788.jpg,500,375
2012_002969.jpg,500,333
2008_000916.jpg,500,375
2012_000065.jpg,500,375


In [31]:
coco_filenames =list(coco_metadata_train.index)

In [33]:
voc_filenames =list(voc_metadata.index)

In [34]:
import os
os.chdir(root_path+"/TinyYOLOv3-Pedestrian-Detection")
os.mkdir("pedestrian_dataset_train")

In [35]:
from shutil import copyfile

def get_class_images(SOURCE, NEW_LOC,CLASS_NAMES):

    names_list =os.listdir(SOURCE)

    for i in names_list:
        if i in CLASS_NAMES:
                copyfile(SOURCE + "/" + i, NEW_LOC + "/" + i)

#dict_pedestrian = get_pedestrian(data) 
#pedestrian_names = list(dict_pedestrian.keys())
#print(len(pedestrian_names))

In [36]:
#get_class_images(root_path+"/COCODataset/train2017",root_path+"/TinyYOLOv3-Pedestrian-Detection/pedestrian_dataset_train",coco_filenames)

In [37]:
#get_class_images(root_path+"/VOC2012/JPEGImages",root_path+"/TinyYOLOv3-Pedestrian-Detection/pedestrian_dataset_train",voc_filenames)

## Creación de los archivos TFRecords

In [38]:
""" Create tfrecords for image data"""

class ImageTFRecordsCreator(object):
    """ 
    Class
    """
    def __init__(self, dataframe_annotations,dataframe_metadata,num_shards = 1000):
        """ 
        Constructor

        Args:
            num_shards (int): number of sharded tfrecord
        """
        self.num_shards = num_shards
        self.df_annotations = dataframe_annotations
        self.df_metadata = dataframe_metadata
        
    def _int64_feature(self, value):

        if not isinstance(value, list):
            value = [value]
        return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

    def _bytes_feature(self, value):
 
        if not isinstance(value, list):
            value = [value]
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
    
    def _read_image_file(self,filename):

        with tf.io.gfile.GFile(filename, 'rb') as f:
            img_raw = f.read() #Lee la imagen directamente en serialized TensorfProto proto
            
        #width = self.df_metadata[self.df_metadata.file_name == filename]["width"].values
        #height = self.df_metadata[self.df_metadata.file_name == filename]["height"].values
        #depth = 3
        bboxes = self.df_annotations[self.df_annotations.file_name ==filename].iloc[:,1:].values
        dummy_box = [[0.4,0.4,0.5,0.5]]
        num_real_boxes = bboxes.shape[0]
        ux= np.repeat(dummy_box,repeats = 20-bboxes.shape[0],axis=0)
        bboxes =np.float32(np.concatenate((bboxes,ux),axis=0))
        bytes_boxes = tf.io.serialize_tensor(bboxes).numpy() #Pasa Transforms a Tensor into a serialized TensorProto proto.


        return img_raw,bytes_boxes,num_real_boxes
        

    # Convert c data to tfrecord
    def _convert_shard_data_to_tfrecord(self, input_files, output_file):
        """ Convert sharded input files to tfrecords
        """
        print('Generating %s' % output_file)
        with tf.io.TFRecordWriter(output_file) as record_writer:
            for input_file in input_files:
                img_raw,bytes_boxes,num_real_boxes= self._read_image_file(input_file) 
                feature={
                        'bboxes': self._bytes_feature(bytes_boxes),
                        'image_raw': self._bytes_feature(tf.compat.as_bytes(img_raw)),
                        'num_real_boxes':self._int64_feature(num_real_boxes)
                        }
                example = tf.train.Example(features=tf.train.Features(feature=feature))
                record_writer.write(example.SerializeToString())

    # Convert data to tfrecord
    def _convert_data_to_tfrecord(self, input_files, output_dir):
        """ convert input image files to tfrecords
          Args:
            input_files (list): List of input images to be converted to tfr
            output_dir (str): Output folder where created tfr files will be kept
          Description:
            Shard input_files if they are above threshold and create those many tfr
        """
        num_input_files = len(input_files)
        for i in tqdm(range(0, num_input_files, self.num_shards)):
            sharded_input_files = input_files[i:i+self.num_shards]
            tfr_file_name = os.path.join(output_dir, 'tfr_'+str(i))
            self._convert_shard_data_to_tfrecord(sharded_input_files, tfr_file_name)

In [39]:
os.chdir(root_path+"/TinyYOLOv3-Pedestrian-Detection/pedestrian_dataset_train")

In [41]:
input_files = metadata_train_df.index.values
tfrecords_outdir = root_path+"/pedestrian_dataset_train_tfr"
num_shards = 1000
tfr_creator = ImageTFRecordsCreator(annotations_train_df,metadata_train_df,num_shards=num_shards)
tfr_creator._convert_data_to_tfrecord(input_files, tfrecords_outdir)

  0%|          | 0/74 [00:00<?, ?it/s]

Generating /tf/home/sergio/Tesis/pedestrian_dataset_train_tfr/tfr_0


  1%|▏         | 1/74 [00:09<11:32,  9.48s/it]

Generating /tf/home/sergio/Tesis/pedestrian_dataset_train_tfr/tfr_1000


  3%|▎         | 2/74 [00:18<11:19,  9.44s/it]

...

 97%|█████████▋| 72/74 [11:15<00:17,  8.50s/it]

Generating /tf/home/sergio/Tesis/pedestrian_dataset_train_tfr/tfr_72000


 99%|█████████▊| 73/74 [11:24<00:08,  8.49s/it]

Generating /tf/home/sergio/Tesis/pedestrian_dataset_train_tfr/tfr_73000


100%|██████████| 74/74 [11:30<00:00,  9.32s/it]


## Lectura de los TFRecords

In [16]:
import tensorflow as tf
import os
import time

In [17]:
image_feature_description = {
    'bboxes': tf.io.FixedLenFeature([], tf.string),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'num_real_boxes':tf.io.FixedLenFeature([], tf.int64),
}

In [18]:
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

#def _parse_image_function(example_proto_list):
#  # Parse the input tf.Example proto using the dictionary above.
#  return tf.io.parse_single_example(example_proto, image_feature_description)

def read_tfr_dataset(tf_records_list_names):
    num_images=0
    raw_image_dataset = tf.data.TFRecordDataset(tf_records_list_names)
    parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

    for image_features in parsed_image_dataset:
        
        bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float64)
        image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
        num_real_boxes = image_features['num_real_boxes']
        

    return None

In [19]:
def preprocessing(image_features):
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(tf.cast(image, tf.float32),size=(416,416))
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,num_real_boxes

In [20]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
filenames = os. listdir()
raw_image_dataset = tf.data.TFRecordDataset(filenames)
parsed_image_dataset = raw_image_dataset.map(_parse_image_function).map(preprocessing,num_parallel_calls=8)

In [21]:
count = 0
inicio = time.time()
for i in parsed_image_dataset.batch(8):
    count+=1
fin = time.time()
print(fin-inicio)
print(count)

53.28766989707947
9213


In [22]:
len(i)

3

In [24]:
i[1]

<tf.Tensor: shape=(2, 20, 4), dtype=float32, numpy=
array([[[0.46441665, 0.36776564, 0.668375  , 0.65540624],
        [0.3905    , 0.40528125, 0.4794375 , 0.53870314],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       , 0.4       , 0.5       , 0.5       ],
        [0.4       ,

In [68]:
os.chdir("/home/sergio/Documents/VOC2012/pedestrian_dataset_voc_tfr")
filenames = os. listdir()

In [53]:
import time
inicio = time.time()
read_tfr_dataset(filenames)
fin = time.time()

print(fin-inicio)

17.129613161087036


In [68]:
#%timeit read_tfr_dataset(filenames)

26.2 s ± 713 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
os.chdir("/home/sergio/Documents/VOC2012/pedestrian_dataset_voc")
filenames = os. listdir()

In [70]:
from skimage.io import imread

def read_dataset(file_names):
    num_images = 0
    for i in filenames:
        #print(i)
        aux = imread(i)
        num_images +=1
    #print(num_images)
    return None

In [71]:
%timeit read_dataset(filenames)

28.8 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Creacion de los TF Records para el Validation set

In [9]:
#Obtenemos el número máximo de objetos en una sola imagen
image_name = coco_annotations_val.file_name.mode().values[0]
max_objects = len(coco_annotations_val[coco_annotations_val.file_name==image_name])
print(max_objects)

14


In [10]:
coco_metadata_val

,width,height
000000532481.jpg,640,426
000000458755.jpg,640,480
000000385029.jpg,640,480
000000311303.jpg,640,427
000000393226.jpg,640,480
...,...,...
000000229358.jpg,640,423
000000098287.jpg,415,640
000000417779.jpg,640,480
000000024567.jpg,480,640


In [11]:
boxes = coco_annotations_val[coco_annotations_val.file_name =="000000532481.jpg"].iloc[:,1:].values
print(boxes)
dummy_box = [[0.4,0.4,0.5,0.5]]
ux= np.repeat(dummy_box,repeats = 20-boxes.shape[0],axis=0)
print(ux)

np.concatenate((boxes,ux),axis=0)

[[0.39190625 0.39497653 0.50145313 0.547277  ]
 [0.44617187 0.86985915 0.46503125 0.91495305]]
[[0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]
 [0.4 0.4 0.5 0.5]]


array([[0.39190625, 0.39497653, 0.50145313, 0.547277  ],
       [0.44617187, 0.86985915, 0.46503125, 0.91495305],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       , 0.5       , 0.5       ],
       [0.4       , 0.4       ,

In [12]:
os.chdir("/home/sergio/TinyYOLOv3-Pedestrian-Detection")
coco_annotations_val.to_csv("annotations_val_df.csv",index=False)
coco_metadata_val.to_csv("metadata_val_df.csv",index=False)

In [13]:
coco_annotations_val

,file_name,xmin,ymin,xmax,ymax
0,000000532481.jpg,0.391906,0.394977,0.501453,0.547277
1,000000532481.jpg,0.446172,0.869859,0.465031,0.914953
2,000000458755.jpg,0.107859,0.078646,0.901687,0.986521
3,000000458755.jpg,0.885938,0.002646,1.000000,0.183604
4,000000458755.jpg,0.921906,0.191021,0.999437,0.393250
...,...,...,...,...,...
10999,000000098287.jpg,0.870193,0.408687,0.882000,0.440844
11000,000000098287.jpg,0.419277,0.389062,0.455422,0.418750
11001,000000417779.jpg,0.736297,0.178083,0.765656,0.267104
11002,000000024567.jpg,0.000000,0.000000,0.876958,0.745188


## Creación de los archivos TFRecords

In [19]:
""" Create tfrecords for image data"""

class ImageTFRecordsCreator(object):
    """ 
    Class
    """
    def __init__(self, dataframe_annotations,dataframe_metadata,num_shards = 1000):
        """ 
        Constructor

        Args:
            num_shards (int): number of sharded tfrecord
        """
        self.num_shards = num_shards
        self.df_annotations = dataframe_annotations
        self.df_metadata = dataframe_metadata
        
    def _int64_feature(self, value):

        if not isinstance(value, list):
            value = [value]
        return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

    def _bytes_feature(self, value):
 
        if not isinstance(value, list):
            value = [value]
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
    
    def _read_image_file(self,filename):

        with tf.io.gfile.GFile(filename, 'rb') as f:
            img_raw = f.read() #Lee la imagen directamente en serialized TensorfProto proto
            
        #width = self.df_metadata[self.df_metadata.file_name == filename]["width"].values
        #height = self.df_metadata[self.df_metadata.file_name == filename]["height"].values
        #depth = 3
        bboxes = self.df_annotations[self.df_annotations.file_name ==filename].iloc[:,1:].values
        dummy_box = [[0.4,0.4,0.5,0.5]]
        num_real_boxes = bboxes.shape[0]
        
        ux= np.repeat(dummy_box,repeats = 20-bboxes.shape[0],axis=0)
        bboxes =np.float32(np.concatenate((bboxes,ux),axis=0))
        bytes_boxes = tf.io.serialize_tensor(bboxes).numpy() #Transforms a Tensor into a serialized TensorProto proto.


        return img_raw,bytes_boxes,num_real_boxes
        

    # Convert c data to tfrecord
    def _convert_shard_data_to_tfrecord(self, input_files, output_file):
        """ Convert sharded input files to tfrecords
        """
        print('Generating %s' % output_file)
        with tf.io.TFRecordWriter(output_file) as record_writer:
            total_boxes = 0
            for input_file in input_files:
                img_raw,bytes_boxes,num_real_boxes= self._read_image_file(input_file) 
                total_boxes+=num_real_boxes
                feature={
                        'bboxes': self._bytes_feature(bytes_boxes),
                        'image_raw': self._bytes_feature(tf.compat.as_bytes(img_raw)),
                        'num_real_boxes':self._int64_feature(num_real_boxes)
                        }
                example = tf.train.Example(features=tf.train.Features(feature=feature))
                record_writer.write(example.SerializeToString())
            print(total_boxes)
    # Convert data to tfrecord
    def _convert_data_to_tfrecord(self, input_files, output_dir):
        """ convert input image files to tfrecords
          Args:
            input_files (list): List of input images to be converted to tfr
            output_dir (str): Output folder where created tfr files will be kept
          Description:
            Shard input_files if they are above threshold and create those many tfr
        """
        num_input_files = len(input_files)
        for i in tqdm(range(0, num_input_files, self.num_shards)):
            
            sharded_input_files = input_files[i:i+self.num_shards]
            tfr_file_name = os.path.join(output_dir, 'tfr_'+str(i))
            self._convert_shard_data_to_tfrecord(sharded_input_files, tfr_file_name)

In [20]:
#Donde están las imagenes del validationset de COCO Dataset
os.chdir("/home/sergio/Documents/COCODataset/pedestrian_dataset_val")

In [21]:
input_files = coco_metadata_val.index.values
tfrecords_outdir = "/home/sergio/Documents/pedestrian_dataset_val_tfr_fixed"
num_shards = 1000
tfr_creator = ImageTFRecordsCreator(coco_annotations_val,coco_metadata_val,num_shards=num_shards)
tfr_creator._convert_data_to_tfrecord(input_files, tfrecords_outdir)

  0%|          | 0/3 [00:00<?, ?it/s]

Generating /home/sergio/Documents/pedestrian_dataset_val_tfr_fixed/tfr_0


 33%|███▎      | 1/3 [00:01<00:02,  1.31s/it]

4132
Generating /home/sergio/Documents/pedestrian_dataset_val_tfr_fixed/tfr_1000


 67%|██████▋   | 2/3 [00:02<00:01,  1.28s/it]

4140
Generating /home/sergio/Documents/pedestrian_dataset_val_tfr_fixed/tfr_2000


100%|██████████| 3/3 [00:03<00:00,  1.13s/it]

2732


In [22]:
4132+4140+2732

11004